# UoA Copcsi 765 Assignemnt 1
Shuo Mao 437681250

In [29]:
# loading data
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score
file_path = 'website-phishing.csv'
web = pd.read_csv(file_path)

bcp = pd.read_csv("BCP.csv")
arrhy = pd.read_csv("arrhythmia.csv")

In [30]:
web.head(3)

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Class
0,-1,1,1,1,-1,-1,-1,-1,-1,1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,-1,1,-1,1,0,-1,-1


In [31]:
bcp.head(3)

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2


In [32]:
arrhy.head(3)

,age,sex,height,weight,QRSduration,PRinterval,Q-Tinterval,Tinterval,Pinterval,QRS,...,chV6_QwaveAmp,chV6_RwaveAmp,chV6_SwaveAmp,chV6_RPwaveAmp,chV6_SPwaveAmp,chV6_PwaveAmp,chV6_TwaveAmp,chV6_QRSA,chV6_QRSTA,class
0,75,0,190,80,91,193,371,174,121,-16,...,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,8
1,56,1,165,64,81,174,401,149,39,25,...,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,6
2,54,0,172,95,138,163,386,185,102,96,...,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,10


By manually brife view the row data of each one, that I discover in the dataset of arrhythmia have missing value repersenting by question mark "?", thus I'll read through all data and replace `?` by `Null` value, so the machineism can process feature.

In [33]:
# Replace '?' with np.nan for the entire DataFrame
arrhy.replace('?', np.nan, inplace=True)
# Convert all columns to numeric, errors='coerce' will convert errors (non-convertible values) to NaN
data = arrhy.apply(pd.to_numeric, errors='coerce')

# Replace NaN values with the mean of their respective columns for the entire DataFrame
data.fillna(data.mean(), inplace=True)
#generate output csv file for following steps
data.to_csv('new_arrhy.csv', index=False, header=True)

# 1: Decision Stump
A decision stump is a decision tree with a depth of 1. It splits the dataset using the best feature based on a criterion like Gini impurity or information gain.

In [34]:
import numpy as np

class DecisionStump:
    def __init__(self):
        # Initialize the decision stump with default values
        self.threshold = None  # The value to decide if a data point goes left or right
        self.feature_index = None  # The index of the feature to split on
        self.left_value = None  # The most common class on the left side of the stump
        self.right_value = None  # The most common class on the right side of the stump
    
    
    def fit(self, X, y):
        best_info_gain = -1  # Initialize the best information gain to a value that will be updated
        # Iterate over all features in the dataset
        for feature_index in range(X.shape[1]): # in this case feature_index repersending columns in row dataset
            thresholds = np.unique(X[:, feature_index])  # Get all unique values of the feature as potential thresholds
            for threshold in thresholds:
                # Calculate the information gain of splitting on this feature at this threshold
                info_gain = self.information_gain(y, X[:, feature_index], threshold)
                if info_gain > best_info_gain:
                    # If this split provides a better information gain, update the stump's parameters
                    best_info_gain = info_gain
                    self.threshold = threshold
                    self.feature_index = feature_index
        
        # After finding the best split, determine the output value (class) for each side of the split
        # This uses the mean of the target values, which implicitly handles binary classification (0 and 1)
        self.left_value = np.round(np.mean(y[X[:, self.feature_index] <= self.threshold]))
        self.right_value = np.round(np.mean(y[X[:, self.feature_index] > self.threshold]))
        
        # Determine the value for the left and right child nodes
        self.left_value = np.round(np.mean(y[X[:, self.feature_index] <= self.threshold]))
        self.right_value = np.round(np.mean(y[X[:, self.feature_index] > self.threshold]))
        
    def predict(self, X):
        # Generate predictions based on the threshold and feature index
        predictions = np.where(X[:, self.feature_index] <= self.threshold, self.left_value, self.right_value)
        return predictions

    
    def information_gain(self, y, feature_column, threshold):
        parent_entropy = self.entropy(y)  # Calculate the entropy before the split
        
        # Create masks to split the data based on the threshold
        left_child_index = feature_column <= threshold
        right_child_index = feature_column > threshold
        
        # Ensure both sides of the split contain samples
        if np.any(left_child_index) and np.any(right_child_index):
            n = len(y)
            n_left = np.sum(left_child_index)  # Number of samples in the left split
            n_right = np.sum(right_child_index)  # Number of samples in the right split
            # Weighted average of child entropies
            child_entropy = (n_left / n) * self.entropy(y[left_child_index]) + (n_right / n) * self.entropy(y[right_child_index])
            info_gain = parent_entropy - child_entropy  # Information gain is reduction in entropy
            return info_gain
        else:
            return 0

    
    def entropy(self, y):
        if np.any(y < 0):  # Handle datasets where the negative class is labeled as -1
            y = np.where(y == -1, 0, 1)
        proportions = np.bincount(y) / len(y)
        return -np.sum([p * np.log2(p) for p in proportions if p > 0])



In [35]:
# Assume the dataset has been loaded into a DataFrame named data
# and the necessary preprocessing has been done.
# Splitting the dataset into features (X) and target (y)
stump = DecisionStump()


X = web.iloc[:, :-1].values# Selecting all expect y(class)
y = web.iloc[:, -1].values# Selecting Class

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=43)


#fit the decision stump

stump.fit(X_train, y_train)

# Use the stump to make predictions
predictions = stump.predict(X_test)
# Optionally, evaluate the stump's performance
accuracy = np.mean(predictions == y_test)

print(f'Accuracy of the decision stump in website-phisihing dataset: {accuracy:.4f}')

web_DS = accuracy

Accuracy of the decision stump in website-phisihing dataset: 0.8960


In [36]:

X = bcp.iloc[:, :-1].values
y = bcp.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=78)
stump.fit(X_train, y_train)
# Use the stump to make predictions
predictions = stump.predict(X_test)
# Optionally, evaluate the stump's performance
accuracy = np.mean(predictions == y_test)

print(f'Accuracy of the decision stump in BCP dataset: {accuracy:.4f}')

bcp_DS  = accuracy

Accuracy of the decision stump in BCP dataset: 0.8978


In [37]:
arrhy_aft = pd.read_csv("new_arrhy.csv")

X= arrhy_aft.iloc[:,:-1].values
y = arrhy_aft.iloc[:,-1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
stump.fit(X_train, y_train)
# Use the stump to make predictions
predictions = stump.predict(X_test)
# Optionally, evaluate the stump's performance
accuracy = np.mean(predictions == y_test)

print(f'Accuracy of the decision stump in arrhythmia dataset: {accuracy:.4f}')
arrh_DS  = accuracy

Accuracy of the decision stump in arrhythmia dataset: 0.0220


After applying the Decision Stump method to each raw dataset, it was discovered that the Decision Stump works effectively for basic binary classification tasks. For example, in the website-phishing dataset, which has just two possible outcomes `(-1, 1)`, and the BCP dataset `(with outcomes 2,4)`, the accuracy rates were pretty high, about `0.89`.This highlights the effectiveness of Decision Stumps in binary classification scenarios. In contrast, the Decision Stump performs substantially worse in non-binary classification cases, such as the arrhythmia dataset, which has `13` alternative outcomes. In this example, the accuracy dropped to a meager `0.022`, indicating a `2%` probability of making a right forecast. This striking disparity demonstrates Decision Stumps' limits in handling increasingly complicated, non-binary categorization tasks."


# 2: Unpruned Decision Tree
An unpruned decision tree grows until all leaves are pure or until every leaf contains a minimum number of samples. Here, you recursively split the nodes based on the best split criterion.

In [38]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        # Node class to store information about each point in the decision tree.
        self.feature_index = feature_index  # Index of the feature used for splitting
        self.threshold = threshold  # Threshold value for the split at this node
        self.left = left  # Reference to the left child node
        self.right = right  # Reference to the right child node
        self.info_gain = info_gain  # Information gain from the split at this node
        self.value = value  # The class value if this is a leaf node, else None



class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        # Initialize the decision tree classifier with stopping conditions.
        self.root = None  # Root node of the tree, initially None
        self.min_samples_split = min_samples_split  # Minimum number of samples required to split a node
        self.max_depth = max_depth  # Maximum depth of the tree

        
    def build_tree(self, dataset, curr_depth=0):
        # Recursively builds the decision tree from the given dataset.
        X, Y = dataset[:,:-1], dataset[:,-1]  # Split dataset into features (X) and target (Y)
        num_samples, num_features = np.shape(X)
        
        # Base case: stop splitting if minimum samples or maximum depth criteria are met
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split["info_gain"] > 0:  # Ensure positive information gain for splitting
                # Recursively build left and right subtrees
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                return Node(best_split["feature_index"], best_split["threshold"], left_subtree, right_subtree, best_split["info_gain"])
        
        return Node(value=self.calculate_leaf_value(Y))  # Leaf node with class prediction

    
    def get_best_split(self, dataset, num_samples, num_features):
        """ This method identifies the optimal feature and threshold for splitting the dataset into two child nodes. It aims to maximize the information gain from the split."""
        # Initialize the dictionary to store the best split found during the process
        best_split = {}
        # Initialize the maximum information gain to a very low number
        max_info_gain = -float("inf")
         # Iterate over all features in the dataset
        for feature_index in range(num_features):
            # Extract all values for the current feature
            feature_values = dataset[:, feature_index]
            # Identify all unique values of the feature to consider as potential thresholds
            possible_thresholds = np.unique(feature_values)
             # Test each threshold to find the best split for the current feature
            for threshold in possible_thresholds:
                # Split the dataset into two parts: one part where the feature values are less than or equal to the threshold
                 # and another part where the feature values are greater than the threshold
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # Ensure both left and right splits contain data
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    # Extract the target values for the whole dataset, and each of the splits
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # Calculate the information gain achieved by the current split
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # If the information gain from the current split is higher than the maximum found so far,
                    # update the best split to this split
                    if curr_info_gain > max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
        return best_split

    
    def split(self, dataset, feature_index, threshold):
        # Creates an empty array for the left dataset. This will hold all rows
        # where the value in the specified feature column is less than or equal
        # to the threshold.
        dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold]) 
        # Similarly, creates an empty array for the right dataset. This will
        # contain all rows where the value in the specified feature column is
        # greater than the threshold.
        dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
        # Returns the two subsets. These will be used as the new datasets for
        # the subsequent recursive calls to build the left and right subtrees,
        # respectively.
        return dataset_left, dataset_right


    def information_gain(self, parent, l_child, r_child, mode="entropy"):  
        # Calculate the proportion of samples that go to the left and right child nodes.
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        # If the mode is set to "gini", use the Gini impurity measure to calculate information gain.
        if mode == "gini":
            # Information gain is the impurity of the parent node minus the weighted sum of the impurity
            # of the left and right child nodes.
            gain = self.gini_index(parent) - (weight_l * self.gini_index(l_child) + weight_r * self.gini_index(r_child))
        else:
            # If the mode is not "gini" (default is "entropy"), use the entropy measure to calculate information gain.
            gain = self.entropy(parent) - (weight_l * self.entropy(l_child) + weight_r * self.entropy(r_child))
        
        return gain

    
    def entropy(self, y):
        # Identify the unique class labels in the dataset.
        class_labels = np.unique(y)
        # Initialize entropy to zero. Entropy will accumulate the disorder measure for each class.
        entropy = 0
        # Iterate over each class label. 
        for cls in class_labels:
            # Calculate the proportion (p_cls) of instances in the dataset belonging to the current class.
            p_cls = len(y[y == cls]) / len(y)
            # Update the entropy using the entropy formula part associated with this class.
            # The formula for entropy of a class is: -p * log2(p), where p is the class proportion.
            entropy += -p_cls * np.log2(p_cls)
        # Return the total entropy, which represents the disorder within the entire dataset.
        return entropy

    
    def gini_index(self, y):      
        # Identify the unique class labels in the dataset.
        class_labels = np.unique(y)
        # Initialize gini to zero. This will accumulate the squared proportion of each class.
        gini = 0
        # Iterate over each class label.
        for cls in class_labels:
            # Calculate the proportion (p_cls) of instances in the dataset belonging to the current class.
            p_cls = len(y[y == cls]) / len(y)
            # Accumulate the square of the proportion of the current class to the gini score.
            gini += p_cls**2
        # Gini impurity is calculated as 1 minus the accumulated score.
        # This transformation ensures that a higher value indicates higher impurity.
        return 1 - gini

        
    def calculate_leaf_value(self, Y):
        # Convert Y to a list if it's not already one. This ensures compatibility with the 'max' function and 'count' method.
        Y = list(Y)
        # Use the 'max' function with a key that counts the occurrences of each class label in Y.
        # This returns the class label that appears most frequently in Y.
        return max(Y, key=Y.count)

    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''     
        if not tree:
            tree = self.root
        if tree.value is not None:
            print(tree.value)
        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        # Ensures that Y is in the correct shape.
        # If Y is a 1-dimensional array (a common format for labels), it reshapes it to a 2-dimensional array with a single column.
        # This uniform shape is necessary for concatenating Y with X.
        if Y.ndim == 1:
            Y = Y.reshape(-1, 1)
        # This combined dataset format is convenient for the subsequent operations of splitting and building the tree.
        dataset = np.concatenate((X, Y), axis=1)
        # Initiates the recursive process of building the decision tree.
        # The build_tree method will be called with the entire dataset,
        # and it will recursively split the dataset on the best features and thresholds,
        # until it meets the stopping criteria to create leaf nodes.
        self.root = self.build_tree(dataset)

    
    def predict(self, X):
        # This line iterates over each example in the dataset X. For each example (x),
        # it calls the make_prediction method, starting from the root of the tree.
        # The result is a list of predictions corresponding to each example in X.
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions

    
    def make_prediction(self, x, tree):
        # Checks if the current node (tree) is a leaf node by verifying if it has a non-None value.
        # Leaf nodes in a decision tree do not have further splits and contain a class value (prediction).
        if tree.value is not None:
            return tree.value  # If it's a leaf node, return the value (class label) of the leaf node.
        # Retrieves the feature value from the instance x for the feature at index tree.feature_index.
        # This value is used to decide whether to move left or right down the tree.
        feature_val = x[tree.feature_index]
        # If the feature value is less than or equal to the threshold of the current node,
        # the method recursively calls itself to move to the left child of the current node.
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            # If the feature value is greater than the threshold of the current node,
            # the method recursively calls itself to move to the right child of the current node.
            return self.make_prediction(x, tree.right)


In [39]:
# loading website-phishing 
file_path = 'website-phishing.csv'
data = pd.read_csv(file_path, skiprows=1, header=None)

In [40]:
#re-assigning data value to web-phishing
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values.reshape(-1, 1)  # Ensure Y is correctly reshaped

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

# Your classifier's fit and predict methods should work as intended if implemented correctly
classifier = DecisionTreeClassifier(min_samples_split=2, max_depth=5)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test) 


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

web_dt  = accuracy

Accuracy: 0.9249208502939846


In this investigation, I used `min_samples_split = 2` for our classification issue, which corresponds to the binary character of our results (true or false). This setting is often beneficial in such situations. For `max_depth`, I chose a depth of 5. The objective for this decision is to create a balance between model complexity and computing efficiency while avoiding overfitting. A tree that is overly deep may result in overfitting, which occurs when the model learns noise from the training data rather than the real signal, as well as considerably increasing computation time. In contrast, a shallow tree, while quicker, may not capture the underlying patterns well, resulting in underfitting.

I ran tests with `max_depth` values of 2, 5, 8, and 12. A depth of two was insufficient to capture the intricacies of the data. While a depth of 8 produced more accurate results, the longer calculating time was a considerable disadvantage. A depth of 12 resulted in overfitting concerns, indicating that the model was too complicated for our data. Thus, a depth of 5 emerged as the best option, giving a reasonable balance of model accuracy and efficiency, ensuring the model is sophisticated enough to discover meaningful patterns without spending too much time on computations or fitting to noise."

In [41]:
#Ploting tree structure of web data
classifier.print_tree()

X_7 <= 0 ? 0.2957943189016128
 left:X_13 <= -1 ? 0.04430056122522666
  left:-1
  right:X_14 <= -1 ? 0.06460174188584966
    left:X_5 <= -1 ? 0.03600565361530961
        left:X_25 <= 0 ? 0.004079756680114899
                left:X_26 <= -1 ? 0.004227856189806201
                                left:-1
                                right:-1
                right:X_27 <= -1 ? 0.013200203001968669
                                left:-1
                                right:-1
        right:1
    right:X_8 <= -1 ? 0.0697549763254604
        left:X_5 <= -1 ? 0.048180652363138676
                left:X_27 <= -1 ? 0.046886240984504945
                                left:-1
                                right:1
                right:1
        right:X_5 <= -1 ? 0.021731666913208436
                left:X_27 <= -1 ? 0.01651908213159048
                                left:-1
                                right:-1
                right:1
 right:X_13 <= -1 ? 0.04845618966120577
  left:X_5 <

In [42]:
# loading BCP
file_path = 'BCP.csv'
data = pd.read_csv(file_path, skiprows=1, header=None)

In [43]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values.reshape(-1, 1)  # Ensure Y is correctly reshaped

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

# Your classifier's fit and predict methods should work as intended if implemented correctly
classifier = DecisionTreeClassifier(min_samples_split=2, max_depth=5)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

bcp_dt  = accuracy

Accuracy: 0.9197080291970803


In [44]:
# ploting tree 
classifier.print_tree()

X_2 <= 2 ? 0.3487699066940749
 left:X_1 <= 6 ? 0.025447176988002626
  left:X_6 <= 4 ? 0.008622488956713158
    left:2
    right:X_1 <= 3 ? 0.3173553719008265
        left:2
        right:X_4 <= 4 ? 0.31999999999999984
                left:4
                right:2
  right:X_6 <= 1 ? 0.20833333333333334
    left:X_3 <= 2 ? 0.4444444444444444
        left:2
        right:4
    right:4
 right:X_3 <= 2 ? 0.0798383566669814
  left:X_1 <= 5 ? 0.3046875
    left:2
    right:4
  right:X_2 <= 4 ? 0.011560234225791152
    left:X_6 <= 3 ? 0.10528573584254902
        left:X_4 <= 3 ? 0.2268518518518518
                left:X_7 <= 3 ? 0.19753086419753085
                                left:2
                                right:4
                right:4
        right:X_5 <= 5 ? 0.016326530612244913
                left:4
                right:X_0 <= 1002945 ? 0.1866666666666665
                                left:2
                                right:4
    right:X_0 <= 242970 ? 0.00241743364913

In [45]:
# loading arrhythmia
file_path = 'new_arrhy.csv'#Notice do not loading original dataset program will crash
data = pd.read_csv(file_path, skiprows=1, header=None)

In [46]:
#Should expect improve than decision stump
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values.reshape(-1, 1)  # Ensure Y is correctly reshaped

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

# Your classifier's fit and predict methods should work as intended if implemented correctly
classifier = DecisionTreeClassifier(min_samples_split=2, max_depth=5)
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

arrh_dt  = accuracy

Accuracy: 0.6703296703296703


In [47]:
# Plotting tree
classifier.print_tree()

X_223 <= 0.9 ? 0.06665457176492173
 left:X_276 <= 0.3 ? 0.060918986699485034
  left:X_256 <= -0.5 ? 0.15560044954165708
    left:X_5 <= 118.0 ? 0.11819021237303784
        left:X_0 <= 17.0 ? 0.4444444444444444
                left:2.0
                right:1.0
        right:2.0
    right:X_4 <= 121.0 ? 0.13284499054820398
        left:X_75 <= 24.0 ? 0.14222962382445148
                left:X_111 <= 0.0 ? 0.15143876337693218
                                left:1.0
                                right:3.0
                right:X_137 <= 32.0 ? 0.26446280991735543
                                left:15.0
                                right:4.0
        right:9.0
  right:X_14 <= 58.0 ? 0.06625289953896918
    left:X_265 <= 0.6 ? 0.1953816237238697
        left:X_255 <= 0.3 ? 0.15570934256055358
                left:X_0 <= 27.0 ? 0.5
                                left:16.0
                                right:10.0
                right:6.0
        right:X_1 <= 0.0 ? 0.3199999999999998

"After implementing the Decision Tree algorithm on each row of data, observations indicate a distinct performance pattern across different types of classification tasks. In simple binary classification scenarios, the improvement in performance is marginal. However, a significant enhancement is observed in non-binary classification tasks, where the accuracy rate dramatically increased from `0.022` to `0.670`. This improvement aligns with expectations, highlighting the Decision Tree's capability to handle complex classification problems with multiple classes more effectively than simpler, binary classification tasks."

# 3: Pruned Decision Tree
Pruning can be done using several strategies like reduced error pruning (post-pruning) or setting a maximum depth (pre-pruning). In a Jupyter notebook, you can extend the unpruned decision tree with pruning capabilities.

In [48]:
 # cross vaildation and return best performance one
def puring(X_temp, y_temp):
        #define defalut values
        default_max_depth = 9
        # key-value storing both depth and outcome
        records = {}
        X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=41)
        # # Define the K-Fold cross-validator
        # kfold = KFold(n_splits=10, shuffle=True, random_state=42)
        # iterater through 
        for i in range(1, default_max_depth ):
            model1 = DecisionTreeClassifier(min_samples_split=2, max_depth=i)
            model1.fit(X_train, y_train)
            y_pred = model1.predict(X_val)
            accuracy = accuracy_score(y_val, y_pred)
            records[i] = accuracy
            # print("depth:", i, "accuracy:", accuracy)
        best_record = max(records, key=records.get)
        print("picking best depth: ", best_record, "with accuracy ", records[best_record])    
        return  best_record

## Purning method : 
In my pruning method, I go through all conceivable tree depths from 1 to 8. The model is trained at each depth, and its performance is analyzed to determine the ideal depth for the best outcomes. This ideal depth is then utilized to fit the final model, ensuring that the tree is neither too shallow (possibly underfitting) nor too deep (to avoid overfitting), resulting in a balance between model complexity and predictive performance.

In [49]:
file_path = 'website-phishing.csv'
data = pd.read_csv(file_path, skiprows=1, header=None)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

# Your classifier's fit and predict methods should work as intended if implemented correctly

classifier = DecisionTreeClassifier(min_samples_split=2, max_depth=puring(X_train, y_train))
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")

web_dt_p  = accuracy

picking best depth:  7 with accuracy  0.9493441881501583
Accuracy: 0.937584803256445


In [50]:
#plotting
classifier.print_tree()

X_7 <= 0 ? 0.2957943189016128
 left:X_13 <= -1 ? 0.04430056122522666
  left:-1
  right:X_14 <= -1 ? 0.06460174188584966
    left:X_5 <= -1 ? 0.03600565361530961
        left:X_25 <= 0 ? 0.004079756680114899
                left:X_26 <= -1 ? 0.004227856189806201
                                left:X_24 <= -1 ? 0.012094317385524134
                                                                left:-1
                                                                right:X_11 <= -1 ? 0.02499504066653435
                                                                                                                                left:1
                                                                                                                                right:-1
                                right:X_19 <= -1 ? 0.0038422776518014096
                                                                left:X_0 <= -1 ? 0.375
                                                            

In [51]:
file_path = 'BCP.csv'
data = pd.read_csv(file_path, skiprows=1, header=None)
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

# Your classifier's fit and predict methods should work as intended if implemented correctly

classifier = DecisionTreeClassifier(min_samples_split=2, max_depth=puring(X_train, y_train))
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")

bcp_dt_p  = accuracy

picking best depth:  3 with accuracy  0.9635036496350365
Accuracy: 0.9343065693430657


In [52]:
classifier.print_tree()

X_2 <= 2 ? 0.3487699066940749
 left:X_1 <= 6 ? 0.025447176988002626
  left:X_6 <= 4 ? 0.008622488956713158
    left:2
    right:X_1 <= 3 ? 0.3173553719008265
        left:2
        right:4
  right:X_6 <= 1 ? 0.20833333333333334
    left:X_3 <= 2 ? 0.4444444444444444
        left:2
        right:4
    right:4
 right:X_3 <= 2 ? 0.0798383566669814
  left:X_1 <= 5 ? 0.3046875
    left:2
    right:4
  right:X_2 <= 4 ? 0.011560234225791152
    left:X_6 <= 3 ? 0.10528573584254902
        left:2
        right:4
    right:X_0 <= 242970 ? 0.002417433649135034
        left:4
        right:4


In [53]:
file_path = 'new_arrhy.csv'
data = pd.read_csv(file_path, skiprows=1, header=None)
X = data.iloc[:, 1:-1].values
y = data.iloc[:, -1].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

# Your classifier's fit and predict methods should work as intended if implemented correctly

classifier = DecisionTreeClassifier(min_samples_split=2, max_depth=puring(X_train, y_train))
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")

arrh_dt_p  = accuracy

picking best depth:  8 with accuracy  0.6703296703296703
Accuracy: 0.6593406593406593


In [54]:
classifier.print_tree()

X_222 <= 0.9 ? 0.06665457176492173
 left:X_275 <= 0.3 ? 0.060918986699485034
  left:X_255 <= -0.5 ? 0.15560044954165708
    left:X_4 <= 118.0 ? 0.11819021237303784
        left:X_1 <= 156.0 ? 0.4444444444444444
                left:2.0
                right:1.0
        right:2.0
    right:X_3 <= 121.0 ? 0.13284499054820398
        left:X_74 <= 24.0 ? 0.14222962382445148
                left:X_110 <= 0.0 ? 0.15143876337693218
                                left:X_11 <= 57.0 ? 0.13228235294117652
                                                                left:X_261 <= -7.3 ? 0.19134948096885818
                                                                                                                                left:X_8 <= -10.0 ? 0.31999999999999995
                                                                                                                                                                                                                                  

After evaluating the results of the aforementioned decision tree code, it is clear that pruning improves speed for both the web-phishing and BCP datasets. However, there is a significant decline in accuracy for the arrhythmia dataset, which might be ascribed to numerous factors:

- The decision tree has the potential to overfit, meaning that the model becomes overly suited to the training data.

- The pruning procedure may mistakenly delete relevant subtrees, resulting in a loss of essential data required for accurate predictions.

- The intrinsic intricacy of the arrhythmia dataset might also be a contributing issue, since it may need a more sophisticated methodology to model adequately.

  

In [55]:
print(web_DS, web_dt, web_dt_p)
print(bcp_DS, bcp_dt, bcp_dt_p)
print(arrh_DS, arrh_dt, arrh_dt_p)

0.8959746720940751 0.9249208502939846 0.937584803256445
0.8978102189781022 0.9197080291970803 0.9343065693430657
0.02197802197802198 0.6703296703296703 0.6593406593406593


After assessing the performance of decision stump, decision tree, and pruned decision tree on three datasets, it was discovered that the models performed the worst on the arrhythmia dataset compared to the other two. A t-test would be ideal for appropriately assessing and quantifying performance differences in the arrhythmia dataset. This statistical test will aid in identifying the importance of the performance disparities seen across the models, particularly for the arrhythmia dataset.

In [56]:
# using cv =3 to save time
def cross_validate_decision_stump(X, y, cv=3):
    # Flatten y to ensure it's a 1D array
    y = y.ravel()
    score = []
    # Initialize KFold cross-validator
    kf = KFold(n_splits=cv, shuffle=True, random_state=42)    
    # Loop over each split
    for train_index, test_index in kf.split(X):
        # Split the data into training and testing sets for the current fold
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Initialize and fit the decision stump on the training data
        stump = DecisionStump()
        stump.fit(X_train, y_train)
        
        # Predict on the testing data
        y_pred = stump.predict(X_test)
        
        # Calculate the accuracy and store it
        accuracy = accuracy_score(y_test, y_pred)
        score.append(accuracy)
    return score




In [57]:
def cross_validate_decision_tree(X, y, cv=3):
    # Initialize KFold cross-validator with 'cv' splits
    kf = KFold(n_splits=cv, shuffle=True, random_state=42)
    score = []
    # Loop over each fold
    for train_index, test_index in kf.split(X):
        # Split data into training and testing sets for the current fold
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Initialize and fit the decision tree classifier
        classifier = DecisionTreeClassifier(min_samples_split=2, max_depth=5)
        classifier.fit(X_train, y_train.ravel())

        # Predict on the test set and calculate accuracy
        y_pred = classifier.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        # Store the accuracy score
        score.append(accuracy)
    return score


In [58]:
def cross_validate_decision_tree_p(X, y, cv=3):
    # Initialize KFold cross-validator with 'cv' splits
    kf = KFold(n_splits=cv, shuffle=True, random_state=42)
    score = []
    # Loop over each fold
    for train_index, test_index in kf.split(X):
        # Split data into training and testing sets for the current fold
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Initialize and fit the decision tree classifier
        classifier = DecisionTreeClassifier(min_samples_split=2, max_depth=puring(X_train, y_train))
        classifier.fit(X_train, y_train.ravel())

        # Predict on the test set and calculate accuracy
        y_pred = classifier.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        score.append(accuracy)
    return score


In [59]:
from scipy.stats import ttest_rel
# to saving times, that I'll using cv where kfold = 3. 
ds_score = []
dt_score =[]
dt_p_score = []

file_path = 'new_arrhy.csv'
data = pd.read_csv(file_path, skiprows=1, header=None)
X = data.iloc[:, 1:-1].values
y = data.iloc[:, -1].values.reshape(-1, 1)
ds_score = cross_validate_decision_stump(X,y)
dt_score = cross_validate_decision_tree(X,y)
# print(ds_score, dt_score)
# Comparing Decision Stump vs. Unpruned Decision Tree
t_stat, p_value = ttest_rel(ds_score, dt_score)
print(f"Paired t-test between Decision Stump and Unpruned DT on Web dataset: p-value = {p_value:.4f}")

dt_p_score = cross_validate_decision_tree_p(X,y)
# Comparing Unpruned Decision Tree vs. Pruned Decision Tree
t_stat, p_value = ttest_rel(dt_score, dt_p_score)
print(f"Paired t-test between Unpruned DT and Pruned DT on Web dataset: p-value = {p_value:.4f}")


Paired t-test between Decision Stump and Unpruned DT on Web dataset: p-value = 0.0093
picking best depth:  6 with accuracy  0.6447368421052632
picking best depth:  4 with accuracy  0.6973684210526315
picking best depth:  4 with accuracy  0.75
Paired t-test between Unpruned DT and Pruned DT on Web dataset: p-value = 0.6206


The p-value for Decison Stump vs. Decison Tree without Purning is 0.0093, which is less than the standard p-value of 0.05. This implies that their accuracy is not only attributable to random fluctuations in the data. It strongly implies that one model outperforms the other in this dataset. According to the accuracy performance of decison stump and decison tree discussed above, the observed performance difference is unlikely to be due to chance; there is a true underlying difference in how these two models predict arrhythmia events.

The p-value for Decison tree without purning and Decison tree with purning is 0.6206, indicating that on the arrhythmia dataset, the variation in performance between these two methods might be due to chance. Also, the p-value is significantly higher than usual (0.05), indicating that there is no significant difference in the efficacy of the Unpruned DT compared to the Pruned DT in this dataset. Essentially, trimming did not result in a statistically significant increase or reduction in model performance.
